In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)


### 1️⃣ Define the shared graph state

In [15]:
from typing import TypedDict, Optional, List

class InboxState(TypedDict):
    message: str
    waiting_hours: int

    intent: Optional[str]
    sentiment: Optional[str]
    sentiment_score: Optional[float]

    priority_score: Optional[int]
    priority_level: Optional[str]

    retrieved_docs: Optional[List[str]]
    suggested_replies: Optional[str]


### 2️⃣ Intent & Sentiment Node

In [16]:
from textblob import TextBlob

def intent_sentiment_node(state: InboxState) -> InboxState:
    message = state["message"]

    polarity = TextBlob(message).sentiment.polarity
    print('polarity',polarity)
    sentiment = (
        "negative" if polarity < -0.2
        else "positive" if polarity > 0.2
        else "neutral"
    )

    prompt = f"""
    Classify the intent of this message.
    Choose ONLY one:
    delivery_issue, refund_request, general_question, feedback

    Message:
    {message}

    Respond with only the intent.
    """

    intent = llm.invoke(prompt).content.strip()

    return {
        **state,
        "intent": intent,
        "sentiment": sentiment,
        "sentiment_score": polarity,
    }


### 3️⃣ Priority Scoring Node 

In [17]:
def priority_node(state: InboxState) -> InboxState:
    score = 0

    if state["sentiment"] == "negative":
        score += 40

    if state["intent"] in ["delivery_issue", "refund_request"]:
        score += 30

    if state["waiting_hours"] > 24:
        score += 20

    level = (
        "HIGH" if score >= 70
        else "MEDIUM" if score >= 40
        else "LOW"
    )

    return {
        **state,
        "priority_score": score,
        "priority_level": level,
    }


### 4️⃣ RAG Retrieval Node 

In [18]:
import sys
from langchain_community.vectorstores import FAISS, Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

docs = [
    "Delivery delays are usually resolved within 48 hours.",
    "Refund requests are processed within 5 business days.",
    "Escalate negative sentiment cases to senior support."
]

if sys.platform == "darwin":
    vector_db = Chroma.from_texts(docs, embeddings)
else:
    vector_db = FAISS.from_texts(docs, embeddings)


### 5️⃣ Suggested Replies Node

In [19]:
def suggested_reply_node(state: InboxState) -> InboxState:
    context = "\n".join(state.get("retrieved_docs", []))

    prompt = f"""
    You are a professional customer support agent.

    Intent: {state["intent"]}
    Sentiment: {state["sentiment"]}

    Knowledge base:
    {context}

    Generate 2 short, polite, professional reply suggestions.
    """

    replies = llm.invoke(prompt).content

    return {
        **state,
        "suggested_replies": replies
    }


In [20]:
def rag_node(state: InboxState) -> InboxState:
    results = vector_db.similarity_search(state["message"], k=2)
    retrieved = [r.page_content for r in results]

    return {
        **state,
        "retrieved_docs": retrieved
    }


### 6️⃣ Build the LangGraph

In [21]:
from langgraph.graph import StateGraph, END

graph = StateGraph(InboxState)

graph.add_node("intent_sentiment", intent_sentiment_node)
graph.add_node("priority", priority_node)
graph.add_node("rag", rag_node)
graph.add_node("suggested_reply", suggested_reply_node)

graph.set_entry_point("intent_sentiment")

graph.add_edge("intent_sentiment", "priority")
graph.add_edge("priority", "rag")
graph.add_edge("rag", "suggested_reply")
graph.add_edge("suggested_reply", END)

app = graph.compile()


### 7️⃣ Run the graph

In [26]:
result = app.invoke({
    "message": "I’ve been waiting 5 days for my delivery. This is unacceptable., This service is terrible and I want a refund.",
    "waiting_hours": 48,
})

print(result)


polarity -1.0
{'message': 'I’ve been waiting 5 days for my delivery. This is unacceptable., This service is terrible and I want a refund.', 'waiting_hours': 48, 'intent': 'refund_request', 'sentiment': 'negative', 'sentiment_score': -1.0, 'priority_score': 90, 'priority_level': 'HIGH', 'retrieved_docs': ['Delivery delays are usually resolved within 48 hours.', 'Delivery delays are usually resolved within 48 hours.'], 'suggested_replies': 'Okay, here are two reply suggestions, keeping in mind the negative sentiment and the intent to request a refund due to a delivery issue. I\'ll focus on acknowledging the problem and offering a solution based on the provided knowledge base before discussing refund options:\n\n**Option 1:**\n\n> "I understand your frustration with the delayed delivery. These situations are certainly upsetting. Our delivery delays are usually resolved within 48 hours. Can we allow that time to pass and investigate further, or would you prefer we discuss refund options no

In [23]:
for key, value in result.items(): # Assuming the dict is stored in a variable named 'result'
    print(f"--- {key.upper()} ---")
    print(value)
    print()

--- MESSAGE ---
I’ve been waiting 5 days for my delivery. This is unacceptable.

--- WAITING_HOURS ---
48

--- INTENT ---
delivery_issue

--- SENTIMENT ---
neutral

--- SENTIMENT_SCORE ---
0.0

--- PRIORITY_SCORE ---
50

--- PRIORITY_LEVEL ---
MEDIUM

--- RETRIEVED_DOCS ---
['Delivery delays are usually resolved within 48 hours.', 'Delivery delays are usually resolved within 48 hours.']

--- SUGGESTED_REPLIES ---
Okay, here are two short, polite, and professional reply suggestions addressing a delivery issue with a neutral sentiment, referencing a 48-hour resolution timeframe, and designed for a customer support agent:

**Option 1:**

> "Thank you for reaching out. We're sorry to hear about the delay with your delivery. We are looking into this and expect to have a resolution for you within 48 hours. We appreciate your patience."

**Option 2:**

> "We understand your concern regarding your delivery. Please be assured that we're actively investigating this issue. We anticipate a resolut